In [5]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

#authenticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="c6fb9fd9-644b-44c5-8e1f-2ea146326c95",
    resource_group_name="Alexander.Dunnett-rg",
    workspace_name="demo-alexplore"
)

In [18]:
#COMPUTE: check if I already have cluster:
ml_client.compute.get('cluster1-alex').state

'Running'

In [32]:
#ENVIRONMENT:
import os
dependencies_dir = "./dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

#writefile:
file_path = './dependencies/conda.yaml'
file_contents = '''
name: model-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - numpy=1.21.2
  - pip=21.2.4
  - scikit-learn=0.24.2
  - scipy=1.7.1
  - pandas>=1.1,<1.2
  - pip:
    - inference-schema[numpy-support]==1.3.0
    - mlflow== 1.26.1
    - azureml-mlflow==1.42.0
    - psutil>=5.8,<5.9
    - tqdm>=4.59,<4.60
    - ipykernel~=6.0
    - matplotlib
'''
with open(file_path, 'w') as file:
    file.write(file_contents)

In [35]:
#register this customer environment (e.g. kernel)
from azure.ai.ml.entities import Environment
custom_env_name = "aml-scikit-learn"
custom_job_env = Environment(
    name=custom_env_name,
    description="Custom environment for Credit Card Defaults job",
    tags={"scikit-learn": "0.24.2"},
    conda_file=os.path.join(dependencies_dir, "conda.yaml"),
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest"
)
custom_job_env = ml_client.environments.create_or_update(custom_job_env)

In [37]:
custom_job_env.description

'Custom environment for Credit Card Defaults job'